In [1]:
from yaml import load, CLoader as Loader

In [69]:
with open('schema.yml') as f:
  output = load(f, Loader=Loader)

[x['columns'] for x in output['models'] if x['name'] == 'heart_rate__history']

[[{'name': 'member_uuid',
   'data_tests': [{'relationships': {'to': "ref('me')",
      'field': 'member_uuid'}}]},
  {'name': 'dlt_parent_id',
   'data_tests': [{'relationships': {'to': "ref('heart_rate')",
      'field': 'dlt_record_id'}}]}]]

In [14]:
from dlt.sources.rest_api import RESTAPIConfig, rest_api_resources

config = {
    "client": {
        "base_url": "",
    },
    "resource_defaults": {
        "primary_key": "id",
        "write_disposition": "replace",
        "table_format": "delta",
    },
    "resources": [
        {
            "name": "me",
            "endpoint": {
                "path": "https://api.orangetheory.io/v1/people/me",
                "paginator": "single_page",
            },
        },
        {
            "name": "bookings",
            "processing_steps": [
                # make sure we have a workout key to resolve further endpoints
            ],
            "endpoint": {
                "data_selector": "items",
                "path": "https://api.orangetheory.io/v1/bookings/me",
                "paginator": "single_page",
                "params": {
                    "include_canceled": "false",
                },
            },
        },
        {
            "name": "performance_summaries",
            "include_from_parent": ["id"],
            "endpoint": {
                "path": "https://api.orangetheory.io/v1/performance-summaries/{id}",
                "paginator": "single_page",
                "params": {
                    "id": {
                        "type": "resolve",
                        "resource": "bookings",
                        "field": "workout.id",
                    },
                },
                "response_actions": [
                    {"status_code": 204, "action": "ignore"}
                ],
            },
        },
        {
            "name": "body_composition",
            "primary_key": "scanResultUUId",
            "endpoint": {
                "path": "https://api.orangetheory.co/member/members/87e005b6-5fe4-4852-8192-63ec26e061e6/body-composition",
                "paginator": "single_page",
            }
        },
        {
            "name": "heart_rate",
            "primary_key": "memberUuid",
            "endpoint": {
                "path": "https://api.yuzu.orangetheory.com/v1/physVars/maxHr/history?memberUuid=87e005b6-5fe4-4852-8192-63ec26e061e6",
                "paginator": "single_page",
                "data_selector": "$",
            },
        },
        {
            "name": "telemetry",
            "primary_key": "classHistoryUuid",
            "include_from_parent": ["id"],
            "endpoint": {
                "path": "https://api.yuzu.orangetheory.com/v1/performance/summary?classHistoryUuid={id}",
                "paginator": "single_page",
                "data_selector": "$",
                "params": {
                    "id": {
                        "type": "resolve",
                        "resource": "bookings",
                        "field": "workout.id",
                    },
                    "maxDataPoints": 720
                },
            },
        },
        # {
        #     "name": "challenges",
        #     "endpoint": {
        #         "path": "https://api.orangetheory.co/challenges/v3.1/member/87e005b6-5fe4-4852-8192-63ec26e061e6",
        #         "paginator": "single_page",
        #         "data_selector": "Dto",
        #         # "response_actions": [add_and_remove_fields],
        #         # "primary_key": "name",
        #     },
        # }
    ],
}

In [15]:
rest_api_resources(config)